In [1]:
import pandas as pd
Electronics=pd.read_csv('/Users/drakaris/Downloads/ratings_Electronics.csv')
print(Electronics.head())
Electronics.columns=['user','item','rating','timestamp']
#print(Electronics.head())
del Electronics['timestamp']
#print(Electronics.head())

Videogames=pd.read_csv('/Users/drakaris/Downloads/ratings_Video_Games.csv')
Videogames.columns=['user','item','rating','timestamp']
#print(Videogames.head())
del Videogames['timestamp']
#print(Videogames.head())
data=Electronics.append(Videogames)
print(data.head())

    AKM1MP6P0OYPR  0132793040  5.0  1365811200
0  A2CX7LUOHB2NDG  0321732944  5.0  1341100800
1  A2NWSAGRHCP8N5  0439886341  1.0  1367193600
2  A2WNBOD3WNDNKT  0439886341  3.0  1374451200
3  A1GI0U4ZRJA8WN  0439886341  1.0  1334707200
4  A1QGNMC6O1VW39  0511189877  5.0  1397433600
             user        item  rating
0  A2CX7LUOHB2NDG  0321732944     5.0
1  A2NWSAGRHCP8N5  0439886341     1.0
2  A2WNBOD3WNDNKT  0439886341     3.0
3  A1GI0U4ZRJA8WN  0439886341     1.0
4  A1QGNMC6O1VW39  0511189877     5.0


In [2]:
data.to_csv('/Users/drakaris/Downloads/Electronics_Video_Games.csv', index=False,header=False)

In [3]:
import pyspark
import pandas as pd
import numpy as np
from pyspark import SparkContext, since
from pyspark.rdd import RDD
import pyspark.sql

from pyspark.mllib.recommendation import ALS
import math

In [4]:
import os
from pyspark import SQLContext
from pyspark import SparkContext 
from pyspark.sql import SQLContext 
sc = pyspark.SparkContext('local')
sqlContext = SQLContext(sc)

In [5]:
ratings = sc.textFile("/Users/drakaris/Downloads/Electronics_Video_Games.csv").map(lambda line: line.split(","))
ratings = ratings.toDF(['userID','itemID','rating'])
ratings.show()


+--------------+----------+------+
|        userID|    itemID|rating|
+--------------+----------+------+
|A2CX7LUOHB2NDG|0321732944|   5.0|
|A2NWSAGRHCP8N5|0439886341|   1.0|
|A2WNBOD3WNDNKT|0439886341|   3.0|
|A1GI0U4ZRJA8WN|0439886341|   1.0|
|A1QGNMC6O1VW39|0511189877|   5.0|
|A3J3BRHTDRFJ2G|0511189877|   2.0|
|A2TY0BTJOTENPG|0511189877|   5.0|
|A34ATBPOK6HCHY|0511189877|   5.0|
| A89DO69P0XZ27|0511189877|   5.0|
| AZYNQZ94U6VDB|0511189877|   5.0|
|A1DA3W4GTFXP6O|0528881469|   5.0|
|A29LPQQDG7LD5J|0528881469|   1.0|
| AO94DHGC771SJ|0528881469|   5.0|
| AMO214LNFCEI4|0528881469|   1.0|
|A28B1G1MSJ6OO1|0528881469|   4.0|
|A3N7T0DY83Y4IG|0528881469|   3.0|
|A1H8PY3QHMQQA0|0528881469|   2.0|
| A2CPBQ5W4OGBX|0528881469|   2.0|
|A265MKAR2WEH3Y|0528881469|   4.0|
|A37K02NKUIT68K|0528881469|   5.0|
+--------------+----------+------+
only showing top 20 rows



In [8]:
ratings1=ratings.select('userID').distinct()
ratings1.show()

+--------------+
|        userID|
+--------------+
| AFS5ZGZ2M3ZGV|
|A18FTRFQQ141CP|
|A2GPNXFUUV51ZZ|
|A3GF7FD6065R2H|
|A16W4IDX9O70NU|
| A4035XND6J8CS|
|A15K7HV1XD6YWR|
|A3FZ6D8NP9775P|
| AA36DB7PNNJP2|
|A3PDGWYC08DXF4|
|A37JOONBIY5POU|
|A29XPB4YTMCH7N|
| AC8WXA642LUCJ|
|A2S7O09DKY30TH|
|A1G37NGYG23QG2|
|  A44UKZE6XEV9|
| AQJQP7RPKQGI9|
|A3BDHHIUI08PXH|
|A1LT2ILVBRDMN2|
|A3SAN0HBKL4O51|
+--------------+
only showing top 20 rows



In [9]:
maxIndex = ratings1.count()
print(maxIndex)

4719719


In [12]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *
#ratings_filename="/Users/drakaris/Downloads/Electronics_Video_Games.csv"
#ratings_df_schema = StructType([StructField('userID', IntegerType()),StructField('movieID', IntegerType()),StructField('rating', DoubleType())])
from pyspark.sql import Row

In [13]:
with_index = ratings1.rdd.zipWithIndex()
distinct_val = with_index.map(lambda x: (Row(userID = x[0][0], id =x[1])))
distinct_val = distinct_val.toDF()

In [14]:
#ratings_df = sqlContext.read.format('com.databricks.spark.csv').options(header=True, inferSchema=False).schema(ratings_df_schema).load(ratings_filename)
#ratings_df = raw_ratings_df.drop('Timestamp')
#ratings_df.cache()
distinct_val.show()

+---+--------------+
| id|        userID|
+---+--------------+
|  0| AFS5ZGZ2M3ZGV|
|  1|A18FTRFQQ141CP|
|  2|A2GPNXFUUV51ZZ|
|  3|A3GF7FD6065R2H|
|  4|A16W4IDX9O70NU|
|  5| A4035XND6J8CS|
|  6|A15K7HV1XD6YWR|
|  7|A3FZ6D8NP9775P|
|  8| AA36DB7PNNJP2|
|  9|A3PDGWYC08DXF4|
| 10|A37JOONBIY5POU|
| 11|A29XPB4YTMCH7N|
| 12| AC8WXA642LUCJ|
| 13|A2S7O09DKY30TH|
| 14|A1G37NGYG23QG2|
| 15|  A44UKZE6XEV9|
| 16| AQJQP7RPKQGI9|
| 17|A3BDHHIUI08PXH|
| 18|A1LT2ILVBRDMN2|
| 19|A3SAN0HBKL4O51|
+---+--------------+
only showing top 20 rows



In [15]:
joined_df = ratings.join(distinct_val, ['userID'], "left")
rdd1 = joined_df.rdd

In [16]:
rdd1.take(10)

[Row(userID='A00229242RO5Y0ANY7P7H', itemID='B0015LURY2', rating='5.0', id=6027),
 Row(userID='A0031944XK4KVC58YJ6I', itemID='B003U8GCJC', rating='5.0', id=11107),
 Row(userID='A0037940AYP2FFFVL9ZX', itemID='B0053QW61M', rating='5.0', id=13811),
 Row(userID='A0061198154UGJDK69CYW', itemID='B0015AARJI', rating='5.0', id=5960),
 Row(userID='A0061198154UGJDK69CYW', itemID='B0018TCB0O', rating='4.0', id=5960),
 Row(userID='A0061198154UGJDK69CYW', itemID='B001S35Y5E', rating='5.0', id=5960),
 Row(userID='A0061198154UGJDK69CYW', itemID='B0015AARJI', rating='5.0', id=5960),
 Row(userID='A0061198154UGJDK69CYW', itemID='B007WRGNQE', rating='4.0', id=5960),
 Row(userID='A0070539117SQ13VTMHY1', itemID='B0018DQYQ2', rating='5.0', id=6233),
 Row(userID='A0070539117SQ13VTMHY1', itemID='B0086SDU3M', rating='4.0', id=6233)]

In [17]:
joined_df.show()

+--------------------+----------+------+-----+
|              userID|    itemID|rating|   id|
+--------------------+----------+------+-----+
|A00229242RO5Y0ANY...|B0015LURY2|   5.0| 6027|
|A0031944XK4KVC58YJ6I|B003U8GCJC|   5.0|11107|
|A0037940AYP2FFFVL9ZX|B0053QW61M|   5.0|13811|
|A0061198154UGJDK6...|B0015AARJI|   5.0| 5960|
|A0061198154UGJDK6...|B0018TCB0O|   4.0| 5960|
|A0061198154UGJDK6...|B001S35Y5E|   5.0| 5960|
|A0061198154UGJDK6...|B0015AARJI|   5.0| 5960|
|A0061198154UGJDK6...|B007WRGNQE|   4.0| 5960|
|A0070539117SQ13VT...|B0018DQYQ2|   5.0| 6233|
|A0070539117SQ13VT...|B0086SDU3M|   4.0| 6233|
|A00798031DT7GARGO...|B008U4PLLG|   4.0|17692|
|A00916831I6AWOTPR...|B00F02UT8G|   1.0|20383|
|A0232309XTWAG6FEGW93|B003C1FC7K|   5.0|10100|
|A0232309XTWAG6FEGW93|B0081RQO1S|   1.0|10100|
|A0233237HQD2KF9H08SE|B002TLTIS8|   2.0| 9149|
|A02458053VE82ICMG...|B004S50WZG|   2.0|13161|
|A02680541GF3IVW82...|B00BUCLVZU|   5.0|19532|
|A029883694JY0A2TISYL|B00415A5QY|   5.0|11571|
|A03032083BKN

In [18]:
ratings2=ratings.select('itemID').distinct()
ratings2.show()

+----------+
|    itemID|
+----------+
|7793224531|
|9966694242|
|9967222247|
|9985975413|
|9990950369|
|B000000O3J|
|B00000J025|
|B00000J0IV|
|B00000J1PX|
|B00000J3Q1|
|B00000J3SV|
|B00000J47G|
|B00000JBK6|
|B00001W0DC|
|B00002S73F|
|B00003CWGY|
|B00003OPEV|
|B00004SC3R|
|B00004TL5X|
|B00004U0EB|
+----------+
only showing top 20 rows



In [19]:
maxIndex = ratings2.count()
print(maxIndex)

525966


In [20]:
with_index = ratings2.rdd.zipWithIndex()
distinct_val1 = with_index.map(lambda x: (Row(itemID = x[0][0], id2 =x[1])))
distinct_val1 = distinct_val1.toDF()


In [21]:
distinct_val1.show()

+---+----------+
|id2|    itemID|
+---+----------+
|  0|7793224531|
|  1|9966694242|
|  2|9967222247|
|  3|9985975413|
|  4|9990950369|
|  5|B000000O3J|
|  6|B00000J025|
|  7|B00000J0IV|
|  8|B00000J1PX|
|  9|B00000J3Q1|
| 10|B00000J3SV|
| 11|B00000J47G|
| 12|B00000JBK6|
| 13|B00001W0DC|
| 14|B00002S73F|
| 15|B00003CWGY|
| 16|B00003OPEV|
| 17|B00004SC3R|
| 18|B00004TL5X|
| 19|B00004U0EB|
+---+----------+
only showing top 20 rows



In [22]:
joined_df1 = joined_df.join(distinct_val1, ['itemID'], "left")
rdd2 = joined_df1.rdd

In [23]:
rdd2.take(10)

[Row(itemID='0439394422', userID='A1TL721YECDIM8', rating='3.0', id=1055480, id2=2485),
 Row(itemID='0439394422', userID='APDCEJMFDO2YT', rating='5.0', id=3305480, id2=2485),
 Row(itemID='7793224531', userID='AO4Q0RPYN3Y1N', rating='5.0', id=1299050, id2=0),
 Row(itemID='7793224531', userID='A2CK2QW8OUZARL', rating='5.0', id=1889139, id2=0),
 Row(itemID='7793224531', userID='A2CZG9I6ERMOFA', rating='4.0', id=2148308, id2=0),
 Row(itemID='7793224531', userID='AO8QRF4F2XXS1', rating='5.0', id=2290394, id2=0),
 Row(itemID='7793224531', userID='A4NLBWDS2USI', rating='5.0', id=2808467, id2=0),
 Row(itemID='7793224531', userID='A2QE3HV9GW264C', rating='5.0', id=2926918, id2=0),
 Row(itemID='7793224531', userID='ACNP9LJ391F3L', rating='4.0', id=3068052, id2=0),
 Row(itemID='7793224531', userID='AN3CN5M3U865J', rating='4.0', id=4175374, id2=0)]

In [24]:
joined_df1.show()

+----------+--------------+------+-------+----+
|    itemID|        userID|rating|     id| id2|
+----------+--------------+------+-------+----+
|0439394422|A1TL721YECDIM8|   3.0|1055480|2485|
|0439394422| APDCEJMFDO2YT|   5.0|3305480|2485|
|7793224531| AO4Q0RPYN3Y1N|   5.0|1299050|   0|
|7793224531|A2CK2QW8OUZARL|   5.0|1889139|   0|
|7793224531|A2CZG9I6ERMOFA|   4.0|2148308|   0|
|7793224531| AO8QRF4F2XXS1|   5.0|2290394|   0|
|7793224531|  A4NLBWDS2USI|   5.0|2808467|   0|
|7793224531|A2QE3HV9GW264C|   5.0|2926918|   0|
|7793224531| ACNP9LJ391F3L|   4.0|3068052|   0|
|7793224531| AN3CN5M3U865J|   4.0|4175374|   0|
|9861064222|A1W80QTUJOH61Y|   4.0| 186478|2486|
|9966694242|A1JSPRHP0LCC0J|   3.0| 306580|   1|
|9966694242|A28IBOLVCSQRLQ|   5.0| 330139|   1|
|9966694242|A3A5A57U9FH8BC|   5.0|3138622|   1|
|9967222247|A3G7814OO44GZ5|   5.0|3397460|   2|
|9985975413|A16JMK7V2IWZ10|   1.0|1959850|   3|
|9985975413| ALWS3UQJ7ETWW|   1.0|2030495|   3|
|9985975413|A3S3NT5VS54FC8|   5.0|367972

In [25]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder


#als=ALS(userCol="userID",itemCol="movieID", ratingCol="rating", coldStartStrategy="drop",nonnegative=True)

In [26]:
joined_df2=joined_df1.drop('itemID')
#joined_df2.show()

In [27]:
joined_df3=joined_df2.drop('userID')
joined_df3.show()

+------+-------+----+
|rating|     id| id2|
+------+-------+----+
|   3.0|1055480|2485|
|   5.0|3305480|2485|
|   5.0|1299050|   0|
|   5.0|1889139|   0|
|   4.0|2148308|   0|
|   5.0|2290394|   0|
|   5.0|2808467|   0|
|   5.0|2926918|   0|
|   4.0|3068052|   0|
|   4.0|4175374|   0|
|   4.0| 186478|2486|
|   3.0| 306580|   1|
|   5.0| 330139|   1|
|   5.0|3138622|   1|
|   5.0|3397460|   2|
|   1.0|1959850|   3|
|   1.0|2030495|   3|
|   5.0|3679723|   3|
|   5.0|3844869|   3|
|   5.0|4293552|   3|
+------+-------+----+
only showing top 20 rows



In [28]:
df=joined_df3.select("id","id2","rating")
df.show()


+-------+----+------+
|     id| id2|rating|
+-------+----+------+
|1055480|2485|   3.0|
|3305480|2485|   5.0|
|1299050|   0|   5.0|
|1889139|   0|   5.0|
|2148308|   0|   4.0|
|2290394|   0|   5.0|
|2808467|   0|   5.0|
|2926918|   0|   5.0|
|3068052|   0|   4.0|
|4175374|   0|   4.0|
| 186478|2486|   4.0|
| 306580|   1|   3.0|
| 330139|   1|   5.0|
|3138622|   1|   5.0|
|3397460|   2|   5.0|
|1959850|   3|   1.0|
|2030495|   3|   1.0|
|3679723|   3|   5.0|
|3844869|   3|   5.0|
|4293552|   3|   5.0|
+-------+----+------+
only showing top 20 rows



In [29]:
#df.save('/Users/drakaris/Downloads/Electronics_Video_Games.csv', 'com.databricks.spark.csv')

In [194]:
from pyspark.mllib.recommendation\
    import ALS,MatrixFactorizationModel, Rating
    
ratings = df.rdd.map(lambda x: Rating(int(x[0]),\
    int(x[1]), float(x[2])))    

train, test = ratings.randomSplit([0.8,0.2])

print(train.count()) #70,005
print(test.count()) #29,995



7321120
1828113


In [202]:
#Need to cache the data to speed up training
train.cache()
#test.cache()

#Setting up the parameters for ALS
rank = 15 # Latent Factors to be made
numIterations = 35# Times to repeat process
regParam = 0.35
#Create the model on the training data
model = ALS.train(train, rank, numIterations, regParam)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52243)
Traceback (most recent call last):
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/site-packages/py4j/java_gateway.py", line 852, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/site-packages/py4j/java_gateway.py", line 990, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:52243)

In [76]:
model.recommendUsers(242,10)

[Rating(user=52519, product=242, rating=4.650234298211485),
 Rating(user=38896, product=242, rating=3.2448196209320255),
 Rating(user=76890, product=242, rating=3.072546772374353),
 Rating(user=12696, product=242, rating=2.9823640814484524),
 Rating(user=70205, product=242, rating=2.9823640814484524),
 Rating(user=18793, product=242, rating=2.978626192256457),
 Rating(user=52725, product=242, rating=2.9113174981089878),
 Rating(user=8839, product=242, rating=2.9113174981089878),
 Rating(user=55479, product=242, rating=2.884237519188793),
 Rating(user=9662, product=242, rating=2.7481616744071085)]

In [77]:
# For User Y Find N Products to Promote
model.recommendProducts(196,10)



[Rating(user=196, product=67, rating=3.6938825940752356),
 Rating(user=196, product=5665, rating=2.9900092053319094),
 Rating(user=196, product=4581, rating=2.6101358684975153),
 Rating(user=196, product=1332, rating=2.5682584930131767),
 Rating(user=196, product=913, rating=2.415789865946774),
 Rating(user=196, product=5729, rating=2.3812850027861714),
 Rating(user=196, product=1828, rating=2.2995355074893835),
 Rating(user=196, product=5713, rating=2.2445142326657948),
 Rating(user=196, product=7105, rating=2.242506947793156),
 Rating(user=196, product=3778, rating=2.2037059013382465)]

In [78]:
#Predict Single Product for Single User
model.predict(196, 242)



-0.657286139974113

In [79]:
# Predict Multi Users and Multi Products
# Pre-Processing
pred_input = train.map(lambda x:(x[0],x[1]))   



In [80]:
# Lots of Predictions
#Returns Ratings(user, item, prediction)
pred = model.predictAll(pred_input) 

In [81]:
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

#Do the actual join
true_pred = true_reorg.join(pred_reorg)

#Need to be able to square root the Mean-Squared Error
from math import sqrt

MSE = true_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)#Results in 0.7629908117414474

In [82]:
print(RMSE)

0.396909609300436


In [83]:
#Test Set Evaluation
#More dense, but nothing we haven't done before
test_input = test.map(lambda x:(x[0],x[1])) 
pred_test = model.predictAll(test_input)
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))
test_pred = test_reorg.join(pred_reorg)
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)#1.0145549956596238

In [84]:
print(test_RMSE)

2.20186591317568
